# Cartesian Slepian Sequences

This notebook is a simplified version of Slepians.jl/Examples/05_Cartesian_Slepians.ipynb

In [1]:
using Slepians, FastGaussQuadrature
using Plots, FFTW

┌ Info: Precompiling Slepians [97caf2bd-9cc0-4738-8821-c8999f46f3c9]
└ @ Base loading.jl:1278
┌ Warning: Package Slepians does not have Statistics in its dependencies:
│ - If you have Slepians checked out for development and have
│   added Statistics as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with Slepians
└ Loading Statistics into Slepians from project dependency, future warnings for Slepians are suppressed.


## 3D Octagonal Prism

Extending into the third dimension, we can add more nodes. As mentioned before, this code will work in higher dimensions as well. 

In [7]:
M = 4; Kp = 10.0; szs = (16,16,16)
int = (51, 51, 51)
nowtv = [FastGaussQuadrature.gausslegendre(nj) for nj in szs]

lbfun(x) = (x >= 0) * max(-sqrt(2) + x, -1) + (x < 0) * max(-sqrt(2) - x, -1)

no    = [(x, lbfun(x)*y, z) for x in nowtv[1][1] for y in nowtv[2][1] for z in nowtv[3][1]];

using Profile

In [8]:
# scatter(no, camera = (60, 60))

In [10]:
# Compute the Slepians on this basis
lvl = 6
maxrank = 128
@time s, sleps = customsleps(M, Kp, szs, no = no, sqwt = givewts(nowtv), int = int, 
    exact = false, lvl = lvl, maxrank = maxrank);

In [11]:
using HDF5

A = collect(reshape(1:120, 15, 8))

h5open("slep100.h5", "w") do file
    write(file, "s", s)  
    write(file, "M", M)  
    write(file, "Kp", Kp)
    write(file, "szs", szs)
    for i in 1:M
        write(file, "slep$i", sleps[i])
    end
end

In [6]:
#=
# To read in the file:
filename = "slep100.h5"
using HDF5 

M = h5open(filename, "r") do file
    read(file, "M")
end

s = h5open(filename, "r") do file
    read(file, "s")
end

sleps = h5open(filename, "r") do file
    map(i -> read(file, "slep$i"), 1:M)
end
=#